In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-282488
Azure region: westeurope
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-282488


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cpu-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

try:
    # Check if cluster already exists
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    # If cluster doesn't exist, create it
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="Standard_D2_V2",
        max_nodes=4
    )
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

print(f'Compute target created: {compute_target.name}')

Creating a new compute target...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Compute target created: cpu-cluster


In [11]:
#from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': uniform(0.5, 1.5),
        '--max_iter': choice(50, 100, 150)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=compute_target,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4
)

In [13]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
print(f"Experiment URL: {hyperdrive_run.get_portal_url()}")
hyperdrive_run.wait_for_completion(show_output=True)

Experiment URL: https://ml.azure.com/runs/HD_07e6a727-02fe-4a9a-a148-1ab8d12a8c56?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-282488/workspaces/quick-starts-ws-282488&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
RunId: HD_07e6a727-02fe-4a9a-a148-1ab8d12a8c56
Web View: https://ml.azure.com/runs/HD_07e6a727-02fe-4a9a-a148-1ab8d12a8c56?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-282488/workspaces/quick-starts-ws-282488&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-06-08T10:30:00.4418245Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-06-08T10:30:00.7128773Z][SCHEDULER][INFO]Scheduling job, id='HD_07e6a727-02fe-4a9a-a148-1ab8d12a8c56_0' 
[2025-06-08T10:30:00.7842062Z][SCHEDULER][INFO]Scheduling job, id='HD_07e6a727-02fe-4a9a-a148-1ab8d12a8c56_1' 
[2025-06-08T10:30:00.7854030Z][SCHEDULER][INFO]Scheduling job, id='HD_07e6a727-02fe-4a9a-a148-1ab8d12a8c56_2' 


{'runId': 'HD_07e6a727-02fe-4a9a-a148-1ab8d12a8c56',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2025-06-08T10:29:58.648185Z',
 'endTimeUtc': '2025-06-08T10:37:35.305116Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '65a9099a-8cee-4a3e-ab5a-2553dafe838f',
  'user_agent': 'python/3.10.11 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.60.0',
  'best_child_run_id': 'HD_07e6a727-02fe-4a9a-a148-1ab8d12a8c56_14',
  'score': '0.9062215477996964',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_07e6a727-02fe-4a9a-a148-1ab8d12a8c56_14'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-

In [17]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

# Print best run details
print(f"Best Run Id: {best_run.id}")
print(f"Best Run Metrics: {best_run.get_metrics()}")
print(f"Best Run Parameters: {best_run.get_details()['runDefinition']['arguments']}")

joblib.dump(best_run.get_metrics(), "./training/hyperdrive_metrics.json")

Best Run Id: HD_07e6a727-02fe-4a9a-a148-1ab8d12a8c56_14
Best Run Metrics: {'Regularization Strength:': 0.9277517986579255, 'Max iterations:': 150, 'Accuracy': 0.9062215477996965}
Best Run Parameters: ['--C', '0.9277517986579255', '--max_iter', '150']


['./training/hyperdrive_metrics.json']

In [18]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [21]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
training_df = x.copy()
training_df['y'] = y

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [22]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy", 
    training_data=training_df,
    label_column_name="y",
    n_cross_validations=5
)

In [23]:
automl_run = exp.submit(automl_config)
automl_run.wait_for_completion(show_output=True)
print("AutoML run has completed!")
best_automl_run = automl_run.get_best_run()
print(f"Best AutoML Run Id: {best_automl_run.id}")

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
Matplotlib is building the font cache; this may take a moment.
2025-06-08 11:03:12.936265: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749380594.303226    4509 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749380594.691852    4509 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749380598.917377    4509 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749380598.917452    4509 computation_placer.cc:1

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_55056829-b896-4757-a061-4fe25b03275e,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2025/06/08 11:20:34 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025-06-08:11:35:13,550 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2025-06-08:11:35:14,576 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2025-06-08:11:35:14,578 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2025-06-08:11:35:15,606 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2025-06-08:

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_55056829-b896-4757-a061-4fe25b03275e,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+------------------------------+--------------------------------+----

AttributeError: 'AutoMLRun' object has no attribute 'get_best_run'

In [24]:
best_automl_run = automl_run.get_best_child()

print(f"Best AutoML Run Id: {best_automl_run.id}")
print(f"Best AutoML Run Metrics: {best_automl_run.get_metrics()}")

Best AutoML Run Id: AutoML_55056829-b896-4757-a061-4fe25b03275e_15
Best AutoML Run Metrics: {'norm_macro_recall': 0.5441083261278353, 'weighted_accuracy': 0.9539516873157148, 'balanced_accuracy': 0.7720541630639177, 'accuracy': 0.9177541729893779, 'AUC_macro': 0.9486845313842996, 'average_precision_score_macro': 0.8295403652433627, 'f1_score_macro': 0.7840817913092906, 'average_precision_score_weighted': 0.9566213832238324, 'f1_score_micro': 0.9177541729893779, 'matthews_correlation': 0.5694336881955456, 'precision_score_macro': 0.7980756144473876, 'f1_score_weighted': 0.9159045610634996, 'precision_score_weighted': 0.9145623321639113, 'AUC_weighted': 0.9486845313842999, 'precision_score_micro': 0.9177541729893779, 'average_precision_score_micro': 0.9818678530308483, 'recall_score_macro': 0.7720541630639177, 'recall_score_micro': 0.9177541729893779, 'log_loss': 0.3560586349037409, 'recall_score_weighted': 0.9177541729893779, 'AUC_micro': 0.9811065093798715, 'confusion_matrix': 'aml://a

In [28]:
# Retrieve and save your best automl model.

best_auto_run, autorun_final_model = automl_run.get_output()

joblib.dump(autorun_final_model, "./training/automl_model.pkl")

['./training/automl_model.pkl']

In [29]:
print(compute_target.get())

compute_target.delete()

{'id': '/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourceGroups/aml-quickstarts-282488/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-282488/computes/cpu-cluster', 'name': 'cpu-cluster', 'type': 'Microsoft.MachineLearningServices/workspaces/computes', 'location': 'westeurope', 'tags': {}, 'properties': {'createdOn': '2025-06-08T09:52:15.6039617+00:00', 'modifiedOn': '2025-06-08T09:52:26.551149+00:00', 'disableLocalAuth': False, 'description': None, 'resourceId': None, 'computeType': 'AmlCompute', 'computeLocation': 'westeurope', 'provisioningState': 'Succeeded', 'provisioningErrors': None, 'provisioningWarnings': {}, 'isAttachedCompute': False, 'properties': {'vmSize': 'Standard_D2_V2', 'vmPriority': 'Dedicated', 'scaleSettings': {'maxNodeCount': 4, 'minNodeCount': 0, 'nodeIdleTimeBeforeScaleDown': 'PT30M'}, 'subnet': None, 'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount':